In [1]:
#Dependencies
import requests
import json
import pandas as pd

#Import API keys
from api_keys import census_key


In [2]:


# API key from Census Bureau
from api_keys import census_key

# Base URL for 2021 ACS data
base_url = "https://api.census.gov/data/2021/acs/acs1"

# Parameters to get city-level population, people below poverty, and median income data
params = {
    "get": "NAME,B01003_001E,B17021_002E,B19013_001E",  # Population, People Below Poverty Level, and Median Income data
    "for": "place:*",                                   # All cities
    "in": "state:*",                                    # All states
    "key": census_key                                   # Your API key
}

# Send GET request to the API
try:
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # Check if request was successful
    
    # Process the data if available
    data = response.json()
    if data and len(data) > 1:
        # Extract headers (columns) and rows
        columns = data[0]
        rows = data[1:]
        
        # Create a DataFrame
        df = pd.DataFrame(rows, columns=columns)
        
        # Rename columns for clarity
        df.rename(columns={
            "NAME": "City",
            "B01003_001E": "Population",
            "B17021_002E": "People Below Poverty Level",
            "B19013_001E": "Median Household Income"
        }, inplace=True)
        
        # Handle missing or None values (replace with 0)
        df["People Below Poverty Level"] = pd.to_numeric(df["People Below Poverty Level"], errors='coerce').fillna(0)
        df["Population"] = pd.to_numeric(df["Population"], errors='coerce').fillna(0)
        df["Median Household Income"] = pd.to_numeric(df["Median Household Income"], errors='coerce').fillna(0)
        
        # Convert columns to integers after cleaning up None values
        df["Population"] = df["Population"].astype(int)
        df["People Below Poverty Level"] = df["People Below Poverty Level"].astype(int)
        df["Median Household Income"] = df["Median Household Income"].astype(int)
        
        # Sort the cities by population in descending order
        df_sorted = df.sort_values(by="Population", ascending=False)
        
        # Calculate poverty percentage (optional)
        df_sorted["Poverty Percentage"] = (df_sorted["People Below Poverty Level"] / df_sorted["Population"]) * 100
        
        # Display top 100 cities by population and include the new median income data
        top_100_cities = df_sorted.head(100)
        print(top_100_cities[[
            "City", "Population", "People Below Poverty Level", "Poverty Percentage", "Median Household Income"
        ]])

    else:
        print("No valid data returned from the Census API.")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data from Census API: {e}")
     

   




                             City  Population  People Below Poverty Level  \
437       New York city, New York     8467513                     1499249   
95   Los Angeles city, California     3849306                      646672   
281        Chicago city, Illinois     2696561                      451479   
535           Houston city, Texas     2287047                      435140   
19          Phoenix city, Arizona     1624539                      239428   
..                            ...         ...                         ...   
272        Boise City city, Idaho      237457                       22480   
591        Norfolk city, Virginia      235089                       31026   
584       Arlington CDP, Virginia      232965                       19353   
612      Spokane city, Washington      229065                       33445   
74       Fremont city, California      227523                       11179   

     Poverty Percentage  Median Household Income  
437           17.705895 